In [1]:
R.<x,y> = PolynomialRing(RR, 2, "xy", order = "lex")

def div_poly(F, f, R = R):
    """
    Implementa el algoritmo de division
    de polinomios multivariados.
    De: Ideals, Varieties, and Algorithms, pag 65
    F es una lista de polinomios, f es el polinomio a dividir
    R es el conjunto de polinomios sobre el que se
    """
    q = [R(0) for _ in range(len(F))]
    r = R(0)
    p = f

    while p!=R(0):
        i=0
        divisionoccurred = False
        while i<len(F) and not divisionoccurred:

            if hasattr(R, "monomial_quotient"):
                quo = R.monomial_quotient(p.lt(),F[i].lt(),coeff=True)
                rem = p.lt().reduce(F[i].lt())
            else:
                quo, rem = p.lt().quo_rem(F[i].lt())
            if rem == 0:
                q[i] = q[i]+quo
                p = p - quo*F[i]
                divisionoccurred = True
            else:
                i += 1
        if not divisionoccurred:
            r = r+p.lt()
            p = p-p.lt()

    return q, r


# Implementando el ejemplo 5 del libro:
Sean $f_1=xy-1$, $f_2=y^2-1$, con órden lexicográfico. Si dividimos $f=xy^2-x$ por $F=(f_1,f_2)$ obtenemos:
$$
xy^2-x=y \cdot (xy-1) + 0\cdot (y^2-1)+(-x+y)
$$

Si tomamos $F=(f_2,f_1)$, obtenemos:
$$
xy^2-x= x\cdot (y^2-1) + 0 \cdot(xy-1) + 0
$$


In [3]:
R.<x,y> = PolynomialRing(RR, 2, "xy", order = "lex")

f = x*y^2-x; f1=x*y-1; f2=y^2-1

[q1,q2], r = div_poly([f1,f2],f)

print("Caso 1:")
show(html(f"${f} = {q1} ({f1}) + {q2} ({f2}) + ({r})$"))


print("Caso 2:")
f = x*y^2-x
f1=x*y-1
f2=y^2-1

[q2,q1], r = div_poly([f2,f1],f)
show(html(f"${f} = {q2} ({f2}) + {q1} ({f1}) + {r}$"))



Caso 1:


\(x*y^2 - x = y (x*y - 1.00000000000000) + 0 (y^2 - 1.00000000000000) + (-x + y)\)

Caso 2:


\(x*y^2 - x = x (y^2 - 1.00000000000000) + 0 (x*y - 1.00000000000000) + 0\)

# Aplicando el algoritmo con polinomios aleatorios

In [223]:
degree=2
n_var = 2
# terms = binomial(degree+n_var, n_var)
terms = 4

R.<x_1,x_2,x_3, x_4> = PolynomialRing(RR, order="lex")
f = R.random_element(degree=degree, terms = terms)

F = [R.random_element(degree=1, terms=2) for _ in range(3)]

Q, r = div_poly(F, f, R = R)

show(html(f"""
$$
\\begin{{align*}}
f = &{f} \\\\
\\sum_i q_i f_i +r = &{
    sum([q*f for q,f in zip(Q,F)]) + r
    }\\\\
    f_1 = &{F[0]} \\quad \\quad q_1 = {Q[0]} \\\\
    f_2 = &{F[1]} \\quad \\quad q_2 = {Q[1]} \\\\
    f_3 = &{F[2]} \\quad \\quad q_3 = {Q[2]} \\\\
    r = &{r}
\\end{{align*}}
$$
"""))


\[ \begin{align*} f = &-0.651549359663330*x_2 + 0.880206541114476*x_3^2 + 0.0442587055060528*x_3 - 0.299907742749156*x_4^2 \\ \sum_i q_i f_i +r = &-0.651549359663330*x_2 + 0.880206541114476*x_3^2 + 0.0442587055060528*x_3 - 0.299907742749156*x_4^2\\     f_1 = &-0.333298817031471*x_2 + 0.0951568793551716*x_3 \quad \quad q_1 = 1.95485050162002 \\     f_2 = &0.594003625357376*x_2 - 0.0638172336413463 \quad \quad q_2 = 0 \\     f_3 = &-0.510738189631297*x_2 - 0.404471935486321 \quad \quad q_3 = 0 \\     r = &0.880206541114476*x_3^2 - 0.141758767834000*x_3 - 0.299907742749156*x_4^2 \end{align*}\]

# Base escalonada

In [340]:

degree = 3
dim = 3

R.<x,y> = PolynomialRing(RR, order = "lex")

V = VectorSpace(R.base_ring(), (degree+1)**2)

polys = [R.random_element(degree = degree) for _ in range(dim)]

vecs = [V([p.coefficient({x: i, y: j}) for i in range(degree+1) for j in range(degree+1)]) for p in polys]

M = matrix(R.base_ring(), vecs)

E=M.echelon_form()

basis = []
for row in E.rows():
    p = sum(row[j]*x^(degree-i)*y^(degree-j) for i in range(degree+1) for j in range(degree+1))
    basis.append(p)

coords = [R.base_ring().random_element() for _ in range(dim)]

coords

poly = sum(basis[i]*coords[i] for i in range(dim))


In [341]:
Q, r = div_poly(basis, poly)

Q, coords


([0.909294046288622, -0.849447801567264, -0.590337792193350],
 [0.909294046288622, -0.849447801567264, -0.590337792193350])